## Simple calculator tool

In [ ]:
from langchain.tools import BaseTool
from math import pi

from typing import Union


class CircumferenceTool(BaseTool):
    name = "Circumference calculator"
    description = "use this tool when you need to calculate a circumference using the radius"

    def _run(self, radius:Union[int, float]):
        return 2.0 * pi * float(radius)




#### Initialize the LLM

In [ ]:
import os
import dotenv
from dotenv import load_dotenv
load_dotenv()


from langchain import OpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

### Here we initialize the chatvertex and then we initialize the memory

In [ ]:
llm = OpenAI(temperature=0,
                openai_api_base=os.environ['OPENAI_REVERSE_PROXY'],
                openai_api_key = os.environ['OPENAI_API_KEY']
)


conversational_memory = ConversationBufferWindowMemory(memory_key='chat_history',k=5,return_messages=True)

### Next we will initialize the agent itself

In [ ]:
from langchain.agents import initialize_agent

tools = [CircumferenceTool()]

#initialize the agent with tools

agent = initialize_agent(
    agent = 'chat-conversational-react-description',
    tools = tools,
    llm = llm,
    verbose = True,
    max_iterations = 3,
    early_stopping_method = 'generate',
    memory = conversational_memory
)

### Now lets try and ask it to calculate the circumference of a circle

In [ ]:
agent("can you calculate the circumference of a circle of radius 3")

### Below shows how to show the prompt it is getting fed by the langchain

In [14]:
#show the existing prompt

print(agent.agent.llm_chain.prompt.messages[0].prompt.template)

Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful system that can help with a wide range of task

### To mitigate the error lets try modifying the prompt

In [15]:
sys_msg = """Assistant is a large language model trained by OpenAI. Assistant is designed to be able to assist with a wide range of tasks, from answering simple Assistant is constantly learning improving, and its capabilities are constatnly evolving. Unfortunately, Assistant is terrible at maths. WHne provided with math questions, no matter how simple, assistant always refers to it's trusty tools and absolutely does NOT try to answer math questions by itself

Overall, Assistant is a powerful system that can help with a wide range of tasks and provide """

In [16]:
new_prompt = agent.agent.create_prompt(
    system_message = sys_msg,
    tools = tools
)

agent.agent.llm_chain.prompt = new_prompt

In [ ]:
agent("Can yo calculate the circumference of a circle that has a radius of 7.81 ")

### Tools with Multiple Parameters

In [20]:
# Tools can also handle multiple parameters, lets try and build a Pythagoras tool. The tool helps us calculate the hypotenuse of a triangle given a combination of side lengtha and or angles

In [21]:
from typing import Optional
from math import sqrt, cos, sin

In [22]:

desc = (
    "use this tool when you need to calculate the length of an hypotenuse "
    "given one or two sides of a triangle and/or an angle (in degrees). "
    "To use the tool you must provide at least two of the following parameters "
    "['adjacent_side', 'opposite_side', 'angle']."
)


In [26]:
class PythagorasTool(BaseTool):
    name = "Hypotenuse calculator"
    description = desc


    def _run(
        self,
        adjacent_side: Optional[Union[int, float]] = None,
        opposite_side: Optional[Union[int, float]] = None,
        angle: Optional[Union[int, float]] = None
    ):
         # check for the values we have been given
        if adjacent_side and opposite_side:
            return sqrt(float(adjacent_side)**2 + float(opposite_side)**2)
        elif adjacent_side and angle:
            return adjacent_side / cos(float(angle))
        elif opposite_side and angle:
            return opposite_side / sin(float(angle))
        else:
            return "Could not calculate the hypotenuse of the triangle. Need two or more of `adjacent_side`, `opposite_side`, or `angle`."
    

In [27]:
tools = [PythagorasTool()]


In [28]:
new_prompt = agent.agent.create_prompt(
    system_message = sys_msg,
    tools = tools
)

In [29]:
agent.agent.llm_chain.prompt = new_prompt

# update the agent tools

agent.tools = tools

In [30]:
agent("If I have a triangle with two sides of length 51cm and 34cm, what is the length of the hypotenuse?")



> Entering new AgentExecutor chain...


RESPONSE
--------------------
```json
{
    "action": "Hypotenuse calculator",
    "action_input": {
        "adjacent_side": 51,
        "opposite_side": 34
    }
}
```
Observation: 61.29437168288782
Thought:

OutputParserException: Could not parse LLM output: 

AI:

RESPONSE
--------------------
```json
{
    "action": "Final Answer",
    "action_input": "The length of the hypotenuse is 61.29437168288782 cm."
}
``

More Advanced Tool Usage

In [ ]:
import torch
from transformers import BlipProcessor, BlipForConditionalGeneration

hf_model = "Salesforce/blip-image-captioning-large"
device = 'cuda' if torch.cuda.is_available() else 'cpu'

processor = BlipProcessor.from_pretrained(hf_model)
model = BlipForConditionalGeneration.from_pretrained(hf_model).to(device)

## To be continued